In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os
import boto3

In [6]:
# # Carregar variáveis de ambiente do arquivo .env
# load_dotenv()

# # Obter variáveis de ambiente
MINIO_ENDPOINT = 'http://minio:9000'
# MINIO_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
# MINIO_SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

In [2]:
s3 = boto3.client('s3', region_name='sa-east-1')  # Specify the region

In [13]:
# Exemplo de leitura/escrita no MinIO
bucket_name = "my-delta-lake"
delta_path = f"s3a://{bucket_name}/delta/sales_data"

response = s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'sa-east-1'}
)

In [9]:
spark = SparkSession.builder \
    .appName("DeltaLakeMinIO") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0,org.apache.hadoop:hadoop-aws:3.2.0") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint",MINIO_ENDPOINT) \
    .getOrCreate()


# Dados de exemplo
data = [("James", "Sales", 3000), ("Michael", "Sales", 4600), ("Robert", "Sales", 4100)]
columns = ["Name", "Department", "Salary"]

# Criação do DataFrame
df = spark.createDataFrame(data, columns)
df.show()

+-------+----------+------+
|   Name|Department|Salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
+-------+----------+------+



In [12]:
local_delta_path = "/tmp/delta-table"
if not os.path.exists(local_delta_path):
    os.makedirs(local_delta_path)

In [ ]:
df.write.format("delta").mode("overwrite").save(local_delta_path)

In [ ]:
def upload_directory_to_s3(local_directory, bucket, s3_directory):
    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_key = os.path.join(s3_directory, relative_path).replace("\\", "/")
            s3.upload_file(local_path, bucket, s3_key)
            print(f"Uploaded {local_path} to s3://{bucket}/{s3_key}")

In [ ]:
s3_path="delta/sales_data"
upload_directory_to_s3(local_delta_path, bucket_name, s3_path)

In [14]:
delta_s3_path = f"s3a://{bucket_name}/{s3_path}"

NameError: name 's3_path' is not defined